# Building a database for Crime Reports

In this project, we will build a database for storing data related with crimes that occurred in Boston. We'll be using Postgres.

### Goals

The goal of this guided project is for you to create a database named `crimes_db` with a table – `boston_crimes` – with appropriate datatypes for storing the data from the `boston.csv` file. 

We will be creating the table inside a schema named `crimes`. 

We will also create the `readonly` and `readwrite` groups with the appropriate privileges as we have discussed in this course. Finally, you will also need to create one user for each of these groups.

<img src="goal.png">

### Creating Crimes Database

Let's start by creating the `crimes_db` database. We'll first connect to the `dq` database with user `dq` when connecting to the postgres server. We'll create our database, then connect to and setup the scheme - `crimes`

In [2]:
import psycopg2

In [71]:
# Connect to postgres database
conn = psycopg2.connect(dbname = "postgres", user = "postgres")
conn.autocommit = True
cur = conn.cursor()

In [ ]:
# Create crimes database
cur.execute("CREATE DATABASE crimes_db;")
conn.autocommit = False

In [72]:
conn.close()

In [2]:
# Connect to our newley created database
conn = psycopg2.connect(dbname = "crimes_db", user = "postgres")
cur = conn.cursor()

In [81]:
# Create a schema within our database

cur.execute("CREATE SCHEMA crimes;")

### Obtaining Column names and sample

Let's take a look at our Boston data

In [3]:
import csv

In [4]:
# Print header and first two rows of our data
with open("Boston.csv", "r") as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)
    print(col_headers)
    print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


### Creating an Auxillary Function

Let's create a function that will tell us all the different values for each column in our data

In [5]:
def get_col_set(csv_filename, col_index):
    values = set()
    with open(csv_filename, 'r') as file:
        next(file)
        reader = csv.reader(file)
        for row in reader:
            values.add(row[col_index])
    return values

In [6]:
print(len(col_headers))

7


In [7]:
for index in range(len(col_headers)):
    values = get_col_set("Boston.csv", index)
    print(f'{col_headers[index]:20} {len(values)}')

incident_number      298329
offense_code         219
description          239
date                 1177
day_of_the_week      7
lat                  18177
long                 18177


### Finding the maximum length

Let's compute the maximum length of each value in the `description` column and `offense_code` column.

In [8]:
description_values = get_col_set("Boston.csv", 2)

max_len = 0
for description in description_values:
    if len(description) > max_len:
        max_len = len(description)
print(max_len)

58


In [9]:
offense_values = get_col_set("Boston.csv", 1)

max_len = 0
for code in offense_values:
    if len(code) > max_len:
        max_len = len(code)
print(max_len)

4


### Creating the Table

Let's now create a table to store the Boston crime data. We can see that for `day_of_the_week` there are 7 unique values. For this, we might want to use an `enumerate datatype`. Let's begin.

In [10]:
print(get_col_set("Boston.csv", 4))

{'Friday', 'Monday', 'Saturday', 'Tuesday', 'Sunday', 'Wednesday', 'Thursday'}


In [82]:
cur.execute("""CREATE TYPE days AS ENUM 
                ('Saturday', 'Wednesday', 'Thursday', 'Sunday', 'Friday', 'Monday', 'Tuesday');""")

In [11]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [12]:
print(first_row)

['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


For these remaining columns, what datatypes should we use?

* `incident_number` - Because numbers go up to 298329, we'll use the `integer` datatype


* `offense_code` - We'll also go with the `smallint` datatype here


* `description` - As this is text, and the maximum length is 58. We'll round up and use the `varchar` datatype with a value of 60.


* `date` - We'll use the `date` type here, which will allow us to perform sane queries.


* `day_of_the_week` - We'll go with our `enumerated data` type. Why? Because because we have a predefined set of values. When we create an enumerated datatype, each possible value is assigned to a 4 byte index within a seperate table. This will ultimately save us memory.


* `lat` - Because we're dealing with decimals, we'll go with the `decimal` datatype


* `long` - Same as above

In [83]:
# Create our boston_crimes table within our new schema
try:
    cur.execute("""CREATE TABLE crimes.boston_crimes (
                incident_number integer PRIMARY KEY,
                 offense_code smallint,
                 description varchar(60),
                 date date,
                 day_of_the_week days,
                 lat decimal,
                 long decimal);""")
except Exception as ex:
    print(ex)

In [13]:
cur.execute("SELECT * FROM crimes.boston_crimes LIMIT 10;")

In [14]:
for i in cur.description:
    print(i)

Column(name='incident_number', type_code=23)
Column(name='offense_code', type_code=21)
Column(name='description', type_code=1043)
Column(name='date', type_code=1082)
Column(name='day_of_the_week', type_code=16468)
Column(name='lat', type_code=1700)
Column(name='long', type_code=1700)


In [15]:
conn.commit()
conn.close()

### Loading the Data

Let's now copy our Boston data into the database. We'll use the `copy_expert` method as below.

In [16]:
# Connect again to our database
conn = psycopg2.connect(dbname = "crimes_db", user = "postgres")
cur = conn.cursor()

In [21]:
with open("Boston.csv", "r") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)

In [17]:
cur.execute("SELECT * FROM pg_user;")

In [18]:
cur.fetchall()

[('postgres', 10, True, True, True, True, '********', None, None),
 ('aaronwright', 16384, True, True, False, False, '********', None, None)]

### Revoking Public Privileges

When we created our database `crimes_db` a `public` schema was automatically created, and grants access to this schema via a role/group `public`. 

We want to revoke all privliges from the `public` group on the `public` schema.

We also want to revoke all privliges of `public` on the `crimes_db` database.

If we don't do this, a user will inherit privileges from the `public` role even if we restrict their privileges and will be able to do stuff with the `public` schema (e.g. create objects)

In [99]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")

In [100]:
cur.execute("REVOKE ALL ON DATABASE crimes_db FROM public;")

### Creating User Groups

Let's now setup some groups. Mainly we want `readonly` and `readwrite` groups setup.

In [26]:
# Create our groups/roles
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("CREATE GROUP readwrite NOLOGIN;")

In [27]:
# Grant our groups/roles connections prviliges to our database
cur.execute("GRANT CONNECT ON DATABASE crimes_db TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crimes_db TO readwrite;")

In [28]:
# Grant usage privilages to our schema
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite")

In [31]:
# Grant prvilieges to our groups
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA CRIMES TO readonly;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA CRIMES TO readwrite;")

In [34]:
# Check our privileges are correct
cur.execute("SELECT grantor, grantee, privilege_type FROM information_schema.table_privileges WHERE table_name = 'boston_crimes';")

In [35]:
cur.fetchall()

[('postgres', 'postgres', 'INSERT'),
 ('postgres', 'postgres', 'SELECT'),
 ('postgres', 'postgres', 'UPDATE'),
 ('postgres', 'postgres', 'DELETE'),
 ('postgres', 'postgres', 'TRUNCATE'),
 ('postgres', 'postgres', 'REFERENCES'),
 ('postgres', 'postgres', 'TRIGGER'),
 ('postgres', 'readonly', 'SELECT'),
 ('postgres', 'readwrite', 'INSERT'),
 ('postgres', 'readwrite', 'SELECT'),
 ('postgres', 'readwrite', 'UPDATE'),
 ('postgres', 'readwrite', 'DELETE')]

### Creating Users

Let's now create one user in each group, giving them a password.

In [36]:
# Set up users
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")

In [37]:
# Assign users to groups
cur.execute("GRANT readonly TO data_analyst;")
cur.execute("GRANT readwrite TO data_scientist;")

In [38]:
conn.commit()
conn.close()

### Testing

Let's now test we've set everything up correctly

In [3]:
conn = psycopg2.connect(dbname = "crimes_db", user = "postgres")

In [4]:
cur = conn.cursor()

In [5]:
# Check our roles have the right prvileges
cur.execute("""SELECT grantee, privilege_type
                FROM information_schema.table_privileges
                WHERE grantee = 'readwrite' OR grantee = 'readonly';""")
cur.fetchall()

[('readonly', 'SELECT'),
 ('readwrite', 'INSERT'),
 ('readwrite', 'SELECT'),
 ('readwrite', 'UPDATE'),
 ('readwrite', 'DELETE')]

In [8]:
# Check all of our existing tables by querying existing tables contained in Postgres engine
cur.execute("SELECT tablename FROM pg_catalog.pg_tables ORDER BY 1;")
cur.fetchall()

[('boston_crimes',),
 ('pg_aggregate',),
 ('pg_am',),
 ('pg_amop',),
 ('pg_amproc',),
 ('pg_attrdef',),
 ('pg_attribute',),
 ('pg_auth_members',),
 ('pg_authid',),
 ('pg_cast',),
 ('pg_class',),
 ('pg_collation',),
 ('pg_constraint',),
 ('pg_conversion',),
 ('pg_database',),
 ('pg_db_role_setting',),
 ('pg_default_acl',),
 ('pg_depend',),
 ('pg_description',),
 ('pg_enum',),
 ('pg_event_trigger',),
 ('pg_extension',),
 ('pg_foreign_data_wrapper',),
 ('pg_foreign_server',),
 ('pg_foreign_table',),
 ('pg_index',),
 ('pg_inherits',),
 ('pg_init_privs',),
 ('pg_language',),
 ('pg_largeobject',),
 ('pg_largeobject_metadata',),
 ('pg_namespace',),
 ('pg_opclass',),
 ('pg_operator',),
 ('pg_opfamily',),
 ('pg_partitioned_table',),
 ('pg_policy',),
 ('pg_proc',),
 ('pg_publication',),
 ('pg_publication_rel',),
 ('pg_range',),
 ('pg_replication_origin',),
 ('pg_rewrite',),
 ('pg_seclabel',),
 ('pg_sequence',),
 ('pg_shdepend',),
 ('pg_shdescription',),
 ('pg_shseclabel',),
 ('pg_statistic',),
 

Here we can see a lot of tables starting with **pg**. These are part of the `pg_catalog` group of tables built into the Postgres engine. So how to we differentiate between tables that are user defined, and those that are inernal?

We can do this by thinking about **schemas**. These are essentially folders for seperating out tables.

When a database is created, 3 schemas are created - `pg_catalog` (for system catalog tables), `information_schema` (for info schema tables), and `public` (the default schemas for creating tables)

When you issue a `CREATE TABLE` command, this will be default be stored under the `public` schema.

So, if we want to see what user created tables have been created, we can filter our above query to exclude tables existing under `pg_catalog` or `information_schema`.

In [17]:
cur.execute("""SELECT tablename, 
                      schemaname 
               FROM pg_catalog.pg_tables 
               WHERE schemaname != 'pg_catalog' AND
                     schemaname != 'information_schema'
               ORDER BY 1;""")
cur.fetchall()

[('boston_crimes', 'crimes')]

In [25]:
cur.execute("SELECT * FROM crimes.boston_crimes LIMIT 10;")
rows = cur.fetchall()
print(type(rows))
for row in rows:
    print(row)

<class 'list'>
(1, 619, 'LARCENY ALL OTHERS', datetime.date(2018, 9, 2), 'Sunday', Decimal('42.35779134'), Decimal('-71.13937053'))
(2, 1402, 'VANDALISM', datetime.date(2018, 8, 21), 'Tuesday', Decimal('42.30682138'), Decimal('-71.06030035'))
(3, 3410, 'TOWED MOTOR VEHICLE', datetime.date(2018, 9, 3), 'Monday', Decimal('42.34658879'), Decimal('-71.07242943'))
(4, 3114, 'INVESTIGATE PROPERTY', datetime.date(2018, 9, 3), 'Monday', Decimal('42.33418175'), Decimal('-71.07866441'))
(5, 3114, 'INVESTIGATE PROPERTY', datetime.date(2018, 9, 3), 'Monday', Decimal('42.27536542'), Decimal('-71.09036101'))
(6, 3820, 'M/V ACCIDENT INVOLVING PEDESTRIAN - INJURY', datetime.date(2018, 9, 3), 'Monday', Decimal('42.29019621'), Decimal('-71.07159012'))
(7, 724, 'AUTO THEFT', datetime.date(2018, 9, 3), 'Monday', Decimal('42.30607218'), Decimal('-71.0827326'))
(8, 3301, 'VERBAL DISPUTE', datetime.date(2018, 9, 3), 'Monday', Decimal('42.32701648'), Decimal('-71.10555088'))
(9, 301, 'ROBBERY - STREET', datet

In [ ]:
conn.close()